In [1]:
# import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import json
import os
import uuid
from PIL import Image
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

In [ ]:
# Training and Validate Sets
degs = [45,90,135,180,225,270,315,360]
pos = 0
positions = ["center", "left-top", "left-center", "left-bottom", "center-top", "center-bottom", "right-top", "right-center", "right-bottom"]
img_bbs = []
count = 0

labels = os.path.join('malaria','training.json')
with open(labels,'r') as json_file:
    datas = json.load(json_file)
    for deg in degs:
        for data in datas:
            img_name = data['image']['pathname']
            names = img_name.split('/')
            read_path = os.path.join('malaria',names[1],names[2])
            img_name_new = "{:010d}".format(count)
            save_path = os.path.join('data','custom','images',img_name_new+".png")
            img = imread(read_path)
            objects = data['objects']
            rects = []
            for obj in objects:
                minr = obj['bounding_box']['minimum']['r']
                minc = obj['bounding_box']['minimum']['c']
                maxr = obj['bounding_box']['maximum']['r']
                maxc = obj['bounding_box']['maximum']['c']
                category = obj['category']
                if (category == "red blood cell" or category == "leukocyte"):
                    box1 = ' '+str(minc)+','+str(minr)+','+str(maxc)+','+str(maxr)+',0'
                    rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=0))
                elif category == "ring" or category == "trophozoite" or category == "schizont" or category == "gametocyte":
                    box1 = ' '+str(minc)+','+str(minr)+','+str(maxc)+','+str(maxr)+',1'
                    rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=1))    
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            image_aug, bbs_aug = iaa.Affine(rotate=deg,mode="reflect")(image=img, bounding_boxes=bbs)
            clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
            imsave(save_path,image_aug)
            h,w,c = image_aug.shape
            print("%d %d" % (w,h))
            first_row_flag = True
            ll = ""
            f = open(os.path.join("data","custom","labels",img_name_new+".txt"),"w")
            for bb in clip_off_bbs.bounding_boxes:
                minx = int(bb.x1)
                miny = int(bb.y1)
                maxx = int(bb.x2)
                maxy = int(bb.y2)
                bb_w = (maxx-minx)/w
                bb_h = (maxy-miny)/h
                bb_xc = (minx + ((maxx-minx)/2))/w
                bb_yc = (miny + ((maxy-miny)/2))/h
                if first_row_flag:
                    ll += str(bb.label)+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
                    first_row_flag = False
                else:
                    ll += "\n"+str(bb.label)+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
            f.write(ll)
            f.close()
            count += 1
            print(deg)
            print(names[2])

In [2]:
# Testing Set

count = 9664

labels = os.path.join('malaria','test.json')
with open(labels,'r') as json_file:
    datas = json.load(json_file)
    for data in datas:
        img_name = data['image']['pathname']
        names = img_name.split('/')
        read_path = os.path.join('malaria',names[1],names[2])
        img_name_new = "{:010d}".format(count)
        save_path = os.path.join('data','custom','images',img_name_new+".jpg")
        img = imread(read_path)
        objects = data['objects']
        h,w,c = img.shape
        print("%d %d" % (w,h))
        first_row_flag = True
        ll = ""
        f = open(os.path.join("data","custom","labels",img_name_new+".txt"),"w")
        for obj in objects:
            miny = int(obj['bounding_box']['minimum']['r'])
            minx = int(obj['bounding_box']['minimum']['c'])
            maxy = int(obj['bounding_box']['maximum']['r'])
            maxx = int(obj['bounding_box']['maximum']['c'])
            bb_w = (maxx-minx)/w
            bb_h = (maxy-miny)/h
            bb_xc = (minx + ((maxx-minx)/2))/w
            bb_yc = (miny + ((maxy-miny)/2))/h
            category = obj['category']
            if (category == "red blood cell" or category == "leukocyte"):
                if first_row_flag:
                    ll += "0"+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
                    first_row_flag = False
                else:
                    ll += "\n"+"0"+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
            elif category == "ring" or category == "trophozoite" or category == "schizont" or category == "gametocyte":
                if first_row_flag:
                    ll += "1"+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
                    first_row_flag = False
                else:
                    ll += "\n"+"1"+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)    
        f.write(ll)
        f.close()
        imsave(save_path,img)
        count += 1
        print(names[2])

1944 1383
41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg
1944 1383
64985a1e-16bb-4016-a01c-c21a5b86e572.jpg
1944 1383
c70894fa-50e8-4eed-a5a8-159a20153a49.jpg
1944 1383
abb72ce2-4193-4dd3-a34c-61c2ee22ccff.jpg
1944 1383
8db76867-671a-4488-a7b6-991e4dd2e05f.jpg
1944 1383
ebc4056e-d766-43cf-b6b4-5dac79d96e84.jpg
1944 1383
d9ecacb0-14c7-4862-930b-18bb51d5f392.jpg
1944 1383
96287315-87a1-4045-9e16-119e045bf12a.jpg
1944 1383
37df4ffb-f268-4557-94d6-bf68d005feae.jpg
1944 1383
8298aca0-5099-4d45-88ca-ec8ac7922e77.jpg
1944 1383
a612e0d5-6a95-43cc-96a6-6fbcb8d45ff4.jpg
1944 1383
a6e72060-5bde-490c-bdd7-7d9cb0fce7fc.jpg
1944 1383
c11081a1-4fd9-4003-8bd9-0727eae62d0a.jpg
1944 1383
738b7be5-59b5-4d8c-8769-cbf2385b9fc7.jpg
1944 1383
51c6bc6b-4f37-4ce3-b624-840b27c1584a.jpg
1944 1383
da2197c7-7911-4be6-8bb9-164f47fd270a.jpg
1944 1383
23e581e7-0fb7-42ff-ba55-a86beefc0619.jpg
1944 1383
54c88d45-30c3-46f8-8fef-b3e9c07781f2.jpg
1944 1383
c3631a04-b757-407b-8e7e-1ed3a6c463d4.jpg
1944 1383
8874ea02-d263-4830-99